## Lab assignment 2

## Optimization in ML (CSL4010)

### **Q1**

In [1]:
import numpy as np
import math
from scipy.optimize import minimize
from cvxopt import solvers, matrix

In [2]:
r = 3

def f(x):
    x1, x2 = x
    return (x1-r)**2 + (x2-r)**2

x0 = [-0.5*r, 1.5*r]

def grad(f, x):
    f1 = f(x)
    h = 1/10**5
    g = np.zeros(len(x))
    for i in range(0, len(x)):
        xx = x.copy()
        xx[i] = xx[i] + h
        g[i] = (f(xx) - f(x))/h

    return g

def grad_hess(f, x):
    n = len(x)
    f0 = f(x)
    h, b, g, H = 1/10**5, np.zeros(n), np.zeros(n), np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x1[i] += h 
        b[i] = f(x1)
        g[i] = (f(x1) - f0)/h 
    
    for i in range(n):
        for j in range(i, n):
            x2 = x.copy()
            x2[i] += h 
            x2[j] += h 
            H[i][j] = (f(x) - b[i] - b[j] + f0)/h**2
            H[j][i] = H[i][j]
    
    return g, H

In [4]:
r = 6
lb = -2*r
ub = 2*r
params = np.array([-0.5*6, 1.5*6])
numParams = len(params)
gradient, hessian = grad_hess(f, params)


lmbda = min(np.linalg.eig(hessian)[0])


if lmbda > 0.01:
    lmbda = 0
else:
    lmbda = 0.01 - lmbda


for i in range(numParams):
    hessian[i,i] += lmbda


A = np.vstack((-np.identity(numParams), np.identity(numParams)))
b = np.append(params - lb, ub - params)
sol = solvers.qp(matrix(hessian, tc = 'd'), matrix(gradient), matrix(A), matrix(b))
print(np.array(sol['x']))


     pcost       dcost       gap    pres   dres
 0: -1.4225e+02 -3.7951e+02  2e+02  7e-17  3e-11
 1: -2.1220e+02 -2.2559e+02  1e+01  2e-16  4e-10
 2: -2.1290e+02 -2.1303e+02  1e-01  1e-16  1e-10
 3: -2.1291e+02 -2.1291e+02  1e-03  3e-16  1e-10
 4: -2.1291e+02 -2.1291e+02  1e-05  2e-16  1e-10
Optimal solution found.
[[  4.95742365]
 [-20.99999993]]


In [5]:
grad(f, x0)

array([-14.99999,  -2.99999])

### **Q2**

In [2]:
import numpy as np
from cvxopt import matrix,solvers


n = 10
I1 = [i for i in range(2, n + 1) if i % 2 == 1]
I2 = [i for i in range(2, n + 1) if i % 2 == 0]


def f1(x):
    x1 = x[0]
    summation = sum((xi - np.sin(6 * np.pi * x1 + i * np.pi / n))**2 for i, xi in enumerate(x[1:], start=2) if i in I1)
    result = x1 + (2 / len(I1)) * summation
    return result


def f2(x):
    x1 = x[0]
    summation = sum((xi - np.sin(6 * np.pi * x1 + i * np.pi / n))**2 for i, xi in enumerate(x[1:], start=2) if i in I2)
    result = 1 - np.sqrt(x1) + (2 / len(I2)) * summation
    return result


def ngrad_hess(func, x):
    h = 1e-5
    g = np.zeros(len(x))
    H = np.zeros((len(x), len(x)))


    for i in range(len(x)):
        x1 = x.copy()
        x1[i] = x1[i] + h
        b = func(x1)
        g[i] = (b - func(x)) / h


    for i in range(len(x)):
        for j in range(len(x)):
            x2 = x.copy()
            x2[i] = x2[i] + h
            x2[j] = x2[j] + h
            H[i, j] = (func(x) - func(x2) - func(x1) + b) / h**2


    return g, H


x = np.random.uniform(-1, 1, n)
x[0] = np.random.uniform(0.001, 1)
result1 = f1(x)
result2 = f2(x)
print("f1(x) =", result1)
print("f2(x) =", result2)


gradient1, hessian1 = ngrad_hess(f1, x)
gradient2, hessian2 = ngrad_hess(f2, x)
print("Gradient of f1(x) =", gradient1)
print("Hessian of f1(x) =", hessian1)
print("Gradient of f2(x) =", gradient2)
print("Hessian of f2(x) =", hessian2)


eigenvalues1, _ = np.linalg.eig(hessian1)
positive_definite1 = all(eigenvalues1 > 0)


min_eigenvalue1 = np.min(eigenvalues1)
if(min_eigenvalue1>0.01):
  lmbda1 = 0
else:
  lmbda1 = 0.01 - min_eigenvalue1


eigenvalues2, _ = np.linalg.eig(hessian2)
positive_definite2 = all(eigenvalues2 > 0)


min_eigenvalue2 = np.min(eigenvalues2)
if(min_eigenvalue2>0.01):
  lmbda2 = 0
else:
  lmbda2 = 0.01 - min_eigenvalue2


lb = np.array([0.001] + [-1.0] * (n - 1))
ub = np.array([1.0] * n)


x1_lb, x1_ub = 0.001, 1
xi_lb, xi_ub = -1, 1
x0 = np.zeros(n)  
x0[0] = np.random.uniform(x1_lb, x1_ub)
x0[1:] = np.random.uniform(xi_lb, xi_ub, size=n - 1)


print("Generated x0:", x0)


P1 = hessian1 + lmbda1*np.identity(n)
q1 = gradient1
P2 = hessian2 + lmbda2*np.identity(n)
q2 = gradient2
G = np.vstack((-np.identity(n), np.identity(n)))
h = np.concatenate((x0 - lb, ub - x0))


if positive_definite1:
    print("Hessian matrix of f1(x) is positive definite.")
else:
    print("Hessian matrix of f1(x) is not positive definite.")


if positive_definite2:
    print("Hessian matrix of f2(x) is positive definite.")
else:
    print("Hessian matrix of f2(x) is not positive definite.")


P1_cvxopt = matrix(P1, tc='d')
q1_cvxopt = matrix(q1, tc='d')
P2_cvxopt = matrix(P2, tc='d')
q2_cvxopt = matrix(q2, tc='d')
G_cvxopt = matrix(G, tc='d')
h_cvxopt = matrix(h, tc='d')


sol1 = solvers.qp(P1_cvxopt, q1_cvxopt, G_cvxopt, h_cvxopt)
sol2 = solvers.qp(P2_cvxopt, q2_cvxopt, G_cvxopt, h_cvxopt)


solution1 = np.array(sol1['x'])
print("Optimal solution:", solution1)


print('Primal Objective for f1(x)')
print(sol1['primal objective'])


solution2 = np.array(sol2['x'])
print("Optimal solution:", solution2)


print('Primal Objective for f2(x)')
print(sol2['primal objective'])

f1(x) = 2.405582196564959
f2(x) = 1.9944975946172545
Gradient of f1(x) = [-10.53084799   0.          -1.02190202   0.          -0.27850238
   0.           1.59863309   0.           0.50026817   0.        ]
Hessian of f1(x) = [[2106152.69794268 1053084.79946498 1155256.38063385 1053084.79946498
  1080918.25076428 1053084.79946498  893212.94951095 1053084.79946498
  1003060.9500955  1053084.79946498]
 [1053084.79946498       0.          102190.20158075       0.
    27850.23784035       0.         -159863.3091815        0.
   -50026.81664035       0.        ]
 [1155256.38063385  102190.20158075  204379.40316587  102190.20158075
   130040.43942111  102190.20158075  -57673.10760074  102190.20158075
    52163.38493597  102190.20158075]
 [1053084.79946498       0.          102190.20158075       0.
    27850.23784035       0.         -159863.3091815        0.
   -50026.81664035       0.        ]
 [1080918.25076428   27850.23784035  130040.43942111   27850.23784035
    55699.47567619   27850.23

### **Q3**

In [2]:
import numpy as np


n = 10
I1 = [i for i in range(2, n + 1) if i % 2 == 1]
I2 = [i for i in range(2, n + 1) if i % 2 == 0]


def f1(x):
    x1 = x[0]
    y_values = []
    for i, xi in enumerate(x[1:], start=2):
        if i in I1:
            y = xi - (0.3 * x1**2 * np.cos(24 * np.pi * x1 + 4 * i * np.pi / n) + 0.6 * x1) * np.cos(6 * np.pi * x1 + i * np.pi / n)
        else:
            y = xi - (0.3 * x1**2 * np.cos(24 * np.pi * x1 + 4 * i * np.pi / n) + 0.6 * x1) * np.sin(6 * np.pi * x1 + i * np.pi / n)
        y_values.append(y)
    summation = sum(y**2 for y in y_values)
    result = x1 + (2 / len(I1)) * summation
    return result


def f2(x):
    x1 = x[0]
    y_values = []
    for i, xi in enumerate(x[1:], start=2):
        if i in I2:
            y = xi - (0.3 * x1**2 * np.cos(24 * np.pi * x1 + 4 * i * np.pi / n) + 0.6 * x1) * np.cos(6 * np.pi * x1 + i * np.pi / n)
        else:
            y = xi - (0.3 * x1**2 * np.cos(24 * np.pi * x1 + 4 * i * np.pi / n) + 0.6 * x1) * np.sin(6 * np.pi * x1 + i * np.pi / n)
        y_values.append(y)
    summation = sum(y**2 for y in y_values)
    result = 1 - np.sqrt(x1) + (2 / len(I2)) * summation
    return result


def ngrad_hess(func, x):
    h = 1e-5
    g = np.zeros(len(x))
    H = np.zeros((len(x), len(x)))


    for i in range(len(x)):
        x1 = x.copy()
        x1[i] = x1[i] + h
        b = func(x1)
        g[i] = (b - func(x)) / h


    for i in range(len(x)):
        for j in range(len(x)):
            x2 = x.copy()
            x2[i] = x2[i] + h
            x2[j] = x2[j] + h
            H[i, j] = (func(x) - func(x2) - func(x1) + b) / h**2


    return g, H


x = np.random.uniform(-1, 1, n)
x[0] = np.random.uniform(0.001, 1)
result1 = f1(x)
result2 = f2(x)
print("f1(x) =", result1)
print("f2(x) =", result2)


gradient1, hessian1 = ngrad_hess(f1, x)
gradient2, hessian2 = ngrad_hess(f2, x)
print("Gradient of f1(x) =", gradient1)
print("Hessian of f1(x) =", hessian1)
print("Gradient of f2(x) =", gradient2)
print("Hessian of f2(x) =", hessian2)




eigenvalues1, _ = np.linalg.eig(hessian1)
positive_definite1 = all(eigenvalues1 > 0)


min_eigenvalue1 = np.min(eigenvalues1)
if(min_eigenvalue1>0.01):
  lmbda1 = 0
else:
  lmbda1 = 0.01 - min_eigenvalue1


eigenvalues2, _ = np.linalg.eig(hessian2)
positive_definite2 = all(eigenvalues2 > 0)


min_eigenvalue2 = np.min(eigenvalues2)
if(min_eigenvalue2>0.01):
  lmbda2 = 0
else:
  lmbda2 = 0.01 - min_eigenvalue2


lb = np.array([0.001] + [-1.0] * (n - 1))
ub = np.array([1.0] * n)


x1_lb, x1_ub = 0.001, 1
xi_lb, xi_ub = -1, 1
x0 = np.zeros(n)  
x0[0] = np.random.uniform(x1_lb, x1_ub)
x0[1:] = np.random.uniform(xi_lb, xi_ub, size=n - 1)


print("Generated x0:", x0)


P1 = hessian1 + lmbda1*np.identity(n)
q1 = gradient1
P2 = hessian2 + lmbda2*np.identity(n)
q2 = gradient2
G = np.vstack((-np.identity(n), np.identity(n)))
h = np.concatenate((x0 - lb, ub - x0))


if positive_definite1:
    print("Hessian matrix of f1(x) is positive definite.")
else:
    print("Hessian matrix of f1(x) is not positive definite.")


if positive_definite2:
    print("Hessian matrix of f2(x) is positive definite.")
else:
    print("Hessian matrix of f2(x) is not positive definite.")
 
P1_cvxopt = matrix(P1, tc='d')
q1_cvxopt = matrix(q1, tc='d')
P2_cvxopt = matrix(P2, tc='d')
q2_cvxopt = matrix(q2, tc='d')
G_cvxopt = matrix(G, tc='d')
h_cvxopt = matrix(h, tc='d')


sol1 = solvers.qp(P1_cvxopt, q1_cvxopt, G_cvxopt, h_cvxopt)
sol2 = solvers.qp(P2_cvxopt, q2_cvxopt, G_cvxopt, h_cvxopt)


solution1 = np.array(sol1['x'])
print("Optimal solution:", solution1)


print('Primal Objective for f1(x)')
print(sol1['primal objective'])


solution2 = np.array(sol2['x'])
print("Optimal solution:", solution2)


print('Primal Objective for f2(x)')
print(sol2['primal objective'])

f1(x) = 2.0025669659830405
f2(x) = 2.1226496420946894
Gradient of f1(x) = [ 9.02434808  0.45372406  0.77560644  0.68066926 -0.39384259 -0.43923391
 -0.37081628  1.01530901 -0.6029721   0.29228268]
Hessian of f1(x) = [[-1.80471469e+06 -9.47806795e+05 -9.79998873e+05 -9.70504980e+05
  -8.63054823e+05 -8.58514811e+05 -8.65353599e+05 -1.00396913e+06
  -8.42134351e+05 -9.31667350e+05]
 [-9.47806795e+05 -9.07458115e+04 -1.22933049e+05 -1.13439332e+05
  -5.98814672e+03 -1.44901479e+03 -8.29077811e+03 -1.46903307e+05
   1.49248039e+04 -7.46006737e+04]
 [-9.79998873e+05 -1.22933049e+05 -1.55122287e+05 -1.45627570e+05
  -3.81763847e+04 -3.36372528e+04 -4.04790161e+04 -1.79091545e+05
  -1.72634341e+04 -1.06788912e+05]
 [-9.70504980e+05 -1.13439332e+05 -1.45627570e+05 -1.36134852e+05
  -2.86826668e+04 -2.41435349e+04 -3.09852982e+04 -1.69597827e+05
  -7.76971618e+03 -9.72951938e+04]
 [-8.63054823e+05 -5.98814672e+03 -3.81763847e+04 -2.86826668e+04
   7.87675181e+04  8.33076500e+04  7.64658867e+04 

### **Q4**

In [2]:
import numpy as np
from cvxopt import matrix,solvers


n = 10


def f1(x):
    n = len(x)
    I1 = [i for i in range(2, n + 1) if i % 2 == 1]
    y = [x[0] - x[0] ** (0.5 * (1 + 3 * (i - 2) / (n - 2))) for i in range(2, n + 1) if i % 2 == 1]
    summation = np.sum(np.square(y))
    product = np.prod(np.cos(20 * np.array(I1) * np.pi / np.sqrt(np.array(I1))))
    return x[0] + (2 / len(I1)) * (4 * summation - 2 * product + 2)


def f2(x):
    n = len(x)
    I2 = [i for i in range(2, n + 1) if i % 2 == 0]
    y = [x[0] - x[0] ** (0.5 * (1 + 3 * (i - 2) / (n - 2))) for i in range(2, n + 1) if i % 2 == 0]
    summation = np.sum(np.square(y))
    product = np.prod(np.cos(20 * np.array(I2) * np.pi / np.sqrt(np.array(I2))))
    return 1 - np.sqrt(x[0]) + (2 / len(I2)) * (4 * summation - 2 * product + 2)


def ngrad_hess(func, x):
    h = 1e-5
    g = np.zeros(len(x))
    H = np.zeros((len(x), len(x)))


    for i in range(len(x)):
        x1 = x.copy()
        x1[i] = x1[i] + h
        b = func(x1)
        g[i] = (b - func(x)) / h
   
    for i in range(len(x)):
        for j in range(len(x)):
            x2 = x.copy()
            x2[i] = x2[i] + h
            x2[j] = x2[j] + h
            H[i, j] = (func(x) - func(x2) - func(x1) + b) / h**2


    return g, H


x = np.random.uniform(-1, 1, n)
x[0] = np.random.uniform(0.001, 1)
result1 = f1(x)
result2 = f2(x)
print("f1(x) =", result1)
print("f2(x) =", result2)


gradient1, hessian1 = ngrad_hess(f1, x)
gradient2, hessian2 = ngrad_hess(f2, x)
print("Gradient of f1(x) =", gradient1)
print("Hessian of f1(x) =", hessian1)
print("Gradient of f2(x) =", gradient2)
print("Hessian of f2(x) =", hessian2)


eigenvalues1, _ = np.linalg.eig(hessian1)
positive_definite1 = all(eigenvalues1 > 0)


min_eigenvalue1 = np.min(eigenvalues1)
if(min_eigenvalue1>0.01):
  lmbda1 = 0
else:
  lmbda1 = 0.01 - min_eigenvalue1


eigenvalues2, _ = np.linalg.eig(hessian2)
positive_definite2 = all(eigenvalues2 > 0)


min_eigenvalue2 = np.min(eigenvalues2)
if(min_eigenvalue2>0.01):
  lmbda2 = 0
else:
  lmbda2 = 0.01 - min_eigenvalue2


lb = np.array([0.001] + [-1.0] * (n - 1))
ub = np.array([1.0] * n)


x1_lb, x1_ub = 0.001, 1
xi_lb, xi_ub = -1, 1
x0 = np.zeros(n)
x0[0] = np.random.uniform(x1_lb, x1_ub)
x0[1:] = np.random.uniform(xi_lb, xi_ub, size=n - 1)


print("Generated x0:", x0)


P1 = hessian1 + lmbda1*np.identity(n)
q1 = gradient1
P2 = hessian2 + lmbda2*np.identity(n)
q2 = gradient2
G = np.vstack((-np.identity(n), np.identity(n)))
h = np.concatenate((x0 - lb, ub - x0))


if positive_definite1:
    print("Hessian matrix of f1(x) is positive definite.")
else:
    print("Hessian matrix of f1(x) is not positive definite.")


if positive_definite2:
    print("Hessian matrix of f2(x) is positive definite.")
else:
    print("Hessian matrix of f2(x) is not positive definite.")


P1_cvxopt = matrix(P1, tc='d')
q1_cvxopt = matrix(q1, tc='d')
P2_cvxopt = matrix(P2, tc='d')
q2_cvxopt = matrix(q2, tc='d')
G_cvxopt = matrix(G, tc='d')
h_cvxopt = matrix(h, tc='d')


sol1 = solvers.qp(P1_cvxopt, q1_cvxopt, G_cvxopt, h_cvxopt)
sol2 = solvers.qp(P2_cvxopt, q2_cvxopt, G_cvxopt, h_cvxopt)


solution1 = np.array(sol1['x'])
print("Optimal solution:", solution1)


print('Primal Objective for f1(x)')
print(sol1['primal objective'])


solution2 = np.array(sol2['x'])
print("Optimal solution:", solution2)


print('Primal Objective for f2(x)')
print(sol2['primal objective'])

f1(x) = 2.056190393355626
f2(x) = 1.190330527662916
Gradient of f1(x) = [0.58301094 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
Hessian of f1(x) = [[-116601.49235482  -58301.09375271  -58301.09375271  -58301.09375271
   -58301.09375271  -58301.09375271  -58301.09375271  -58301.09375271
   -58301.09375271  -58301.09375271]
 [ -58301.09375271       0.               0.               0.
        0.               0.               0.               0.
        0.               0.        ]
 [ -58301.09375271       0.               0.               0.
        0.               0.               0.               0.
        0.               0.        ]
 [ -58301.09375271       0.               0.               0.
        0.               0.               0.               0.
        0.               0.        ]
 [ -58301.09375271       0.               0.               0.
        0.               0.               0.               0.
        0.          

### **Q5**

In [3]:
import numpy as np
from cvxopt import matrix,solvers
n = 10


def f1(x):
    n = len(x)
    I1 = [i for i in range(2, n + 1) if i % 2 == 1]
    y = [x[0] - np.sin(6 * np.pi * x[0] + i * np.pi / n) for i in range(2, n + 1) if i % 2 == 1]
    summation = np.sum(np.square(y))
    return x[0] ** 0.2 + (2 / len(I1)) * summation


def f2(x):
  n = len(x)
  I2 = [i for i in range(2, n + 1) if i % 2 == 0]
  y = [x[0] - np.sin(6 * np.pi * x[0] + i * np.pi / n) for i in range(2, n + 1) if i % 2 == 0]
  summation = np.sum(np.square(y))
  return 1 - x[0] ** 0.2 + (2 / len(I2)) * summation


def ngrad_hess(func, x):
    h = 1e-5
    g = np.zeros(len(x))
    H = np.zeros((len(x), len(x)))


    for i in range(len(x)):
        x1 = x.copy()
        x1[i] = x1[i] + h
        b = func(x1)
        g[i] = (b - func(x)) / h


    for i in range(len(x)):
        for j in range(len(x)):
            x2 = x.copy()
            x2[i] = x2[i] + h
            x2[j] = x2[j] + h
            H[i, j] = (func(x) - func(x2) - func(x1) + b) / h**2


    return g, H


x = np.random.uniform(-1, 1, n)
x[0] = np.random.uniform(0.001, 1)
result1 = f1(x)
result2 = f2(x)
print("f1(x) =", result1)
print("f2(x) =", result2)


gradient1, hessian1 = ngrad_hess(f1, x)
gradient2, hessian2 = ngrad_hess(f2, x)
print("Gradient of f1(x) =", gradient1)
print("Hessian of f1(x) =", hessian1)
print("Gradient of f2(x) =", gradient2)
print("Hessian of f2(x) =", hessian2)
eigenvalues1, _ = np.linalg.eig(hessian1)
positive_definite1 = all(eigenvalues1 > 0)


min_eigenvalue1 = np.min(eigenvalues1)
if(min_eigenvalue1>0.01):
  lmbda1 = 0
else:
  lmbda1 = 0.01 - min_eigenvalue1


eigenvalues2, _ = np.linalg.eig(hessian2)
positive_definite2 = all(eigenvalues2 > 0)


min_eigenvalue2 = np.min(eigenvalues2)
if(min_eigenvalue2>0.01):
  lmbda2 = 0
else:
  lmbda2 = 0.01 - min_eigenvalue2


lb = np.array([0.001] + [-1.0] * (n - 1))
ub = np.array([1.0] * n)


x1_lb, x1_ub = 0.001, 1
xi_lb, xi_ub = -1, 1
x0 = np.zeros(n)
x0[0] = np.random.uniform(x1_lb, x1_ub)
x0[1:] = np.random.uniform(xi_lb, xi_ub, size=n - 1)


print("Generated x0:", x0)


P1 = hessian1 + lmbda1*np.identity(n)
q1 = gradient1
P2 = hessian2 + lmbda2*np.identity(n)
q2 = gradient2
G = np.vstack((-np.identity(n), np.identity(n)))
h = np.concatenate((x0 - lb, ub - x0))


if positive_definite1:
    print("Hessian matrix of f1(x) is positive definite.")
else:
    print("Hessian matrix of f1(x) is not positive definite.")


if positive_definite2:
    print("Hessian matrix of f2(x) is positive definite.")
else:
    print("Hessian matrix of f2(x) is not positive definite.")


P1_cvxopt = matrix(P1, tc='d')
q1_cvxopt = matrix(q1, tc='d')
P2_cvxopt = matrix(P2, tc='d')
q2_cvxopt = matrix(q2, tc='d')
G_cvxopt = matrix(G, tc='d')
h_cvxopt = matrix(h, tc='d')


sol1 = solvers.qp(P1_cvxopt, q1_cvxopt, G_cvxopt, h_cvxopt)
sol2 = solvers.qp(P2_cvxopt, q2_cvxopt, G_cvxopt, h_cvxopt)


solution1 = np.array(sol1['x'])
print("Optimal solution:", solution1)


print('Primal Objective for f1(x)')
print(sol1['primal objective'])


solution2 = np.array(sol2['x'])
print("Optimal solution:", solution2)


print('Primal Objective for f2(x)')
print(sol2['primal objective'])

f1(x) = 1.3620930365807982
f2(x) = 0.6527263476097187
Gradient of f1(x) = [-16.91513881   0.           0.           0.           0.
   0.           0.           0.           0.           0.        ]
Hessian of f1(x) = [[3382352.64556452 1691513.88060618 1691513.88060618 1691513.88060618
  1691513.88060618 1691513.88060618 1691513.88060618 1691513.88060618
  1691513.88060618 1691513.88060618]
 [1691513.88060618       0.               0.               0.
        0.               0.               0.               0.
        0.               0.        ]
 [1691513.88060618       0.               0.               0.
        0.               0.               0.               0.
        0.               0.        ]
 [1691513.88060618       0.               0.               0.
        0.               0.               0.               0.
        0.               0.        ]
 [1691513.88060618       0.               0.               0.
        0.               0.               0.               0